In [2]:
import pandas as pd
import getpass
from sqlalchemy import create_engine
import MySQLdb

# Kāpēc ir vajadzīga datubāze?  
<!-- e-veikala piemērs. Clients, Sales, Products -->
* Kāpēc neglabāt visu uzņēmuma informāciju vienā lielā ekseļa tabulā? Piem. visi klienti un preču nosaukumi kolonnās, tāpat kā ir vai nav piegādāts! Viss vienkārši!


In [3]:
dct = {"klients kurs vakar zvanija": 
       {"klienta_vards-uzvards":"Andris Berzins", 
       "klienta_tel_nr" : "+371 12345678", 
       "nopirkta_prece": "laikam kkads samsung ledusskapis vins teica pa telefonu",
       "piegadats": "ne",
        "kurjers": "Andrejs"
       }, 
       "klients kurs daudz sudzas":
       {"klienta_vards-uzvards":"Peteris Kalnins", 
       "klienta_tel_nr": "+371 87654321",
       "nopirkta_prece": "Huwaei S11",
       "piegadats":"ja",
       "kurjers": "Raivis"
       },
       "vel viens klients":
       {"klienta_vards-uzvards":"Andris Berzins", 
       "klienta_tel_nr": "+371 89012345",
       "nopirkta_prece": "nezinams",
       "piegadats":"??",
       "kurjers": "Edgars"
       } 
      }
tabula = pd.DataFrame(dct).T

In [4]:
tabula

,klienta_vards-uzvards,klienta_tel_nr,nopirkta_prece,piegadats,kurjers
klients kurs vakar zvanija,Andris Berzins,+371 12345678,laikam kkads samsung ledusskapis vins teica pa...,ne,Andrejs
klients kurs daudz sudzas,Peteris Kalnins,+371 87654321,Huwaei S11,ja,Raivis
vel viens klients,Andris Berzins,+371 89012345,nezinams,??,Edgars


Problēmas, kuras mēs varētu gribēt atrisināt izmantojot mums pieejamos datus:  
* Ja nu mēs gribam apskatīt visas Andra Bērziņa pirktās preces?  
* Ja nu mums ir jāveic noliktavas inventarizācija un jāskaita cik televizori un cik datori ir pārdoti?  
* Ja nu noliktavā krājās preces, kuras netiek pārdotas un mēs gribētu saprast kuras tieši netiek pārdotas visvairāk relatīvi pret iepirkto daudzumu?  
* Ja nu klienti dažreiz sūdzās par piegādes kvalitāti un mēs gribam saprast kurš kurjers ir atbildīgs?  
* Ja nu mums mainās preču katalogs vai mums ir trīs dažādi samsung ledusskapji?  
* Ja nu klients vēlās uzzināt kāpēc preces nav, bet tabulā rakstīts, ka ir piegādāts?  
* Ja nu mēs atrast mūsu aktīvākos klientus (Klientus, kuri nopirkuši vairāk kā trīs preces) ?

## Vienkāršākie tabulu noteikumi / labās prakses:
* **Visai informācijai ir jābūt attēlotai tabulās**
* **Kolonnu un rindiņu secībai nedrīkst būt nozīmei**
* **Trūkstošām vērtībām jābūt apzīmētām vienādi visur**
* **Visiem ierakstiem katrā tabulā jābūt unikāli un universāli indeksētiem vai nu pēc vienas vai vairākām pazīmēm**
* **Kolonnu nosaukumiem vienas tabulas ietvaros jābūt unikāliem**
* **Tabulām ir jāsatur vai nu informāciju (dimensiju tabulas) vai arī atsauces uz citām tabulām (faktu tabulas)**

Uzdevums:  
Izdomājiet kādas tabulas veidot, ievērojot tabulu noteikumus, lai varētu atbildēt uz visām uzdotajām problēmām

Jautājumi:
* **Kāpēc neglabāt šīs te tabulas vienā vai vairākos excel failos?**  
<!-- 1. Jo mēs gribētu šai informācijai efektīvi piekļūt. Atlasīt no vienas tabulas šķēlumu, kas atbilst citas tabulas šķēlumam ir laikietilpīgi, lai uzrakstītu un arī lai izpildītu 2. Jo liels ekselis var būt ļoti lēns 3. Mēs varētu gribētu šim ekselim piekļūt no vairākām vietām vienlaicīgi un paralēli (piem vienlaikus kurjers atzīmē, ka ir piegādāta prece no telefona aplikācijas un mums parādās jauns klients, kurš nopērk preci, kas ir ienākošs pieprasījums no mājaslapas, varbūt noliktavā ienāk sūtījums un darbinieks manuāli liek klāt jaunas preces)-->

* **Kāpēc neglabāt šīs tabulas .csv failos?**
<!-- Nav slikta ideja, lai atrisinātu liela ekseļa lēnuma problēmu, bet tas neatrisina nepieejamību. .csv fails arī nozīmē, ka mums būtu jāuzraksta kods, kas šos failus ielasa, apstrādā un saglabā-->

# Kas ir datubāze?  
Datubāze ir strukturētu datu kopa, kas glabājas kādā datorā un ir pieejama / apskatāma / rediģējama vairākos veidos  

Datubāze ar kuru mēs strādāsim ir relāciju datubāze. Relāciju datubāze atbalsta relāciju modeli, kura vienkāršota versija ir jau minētie tabulu nosaukumi. Īsi sakot, tabulas var 'savienot' citu ar citu, izmantojot tabulu ierakstiem piemītošos unikālos indeksus, sauktus arī par atslēgām.

Datubāzu tabulām piemīt tā saucamie stingrie tipi. Tabulu veidojot, kolonnām tiek piešķirti nosaukumi un tipi, kuri tajās glabāsies.



Katrā tabulā tai piemītošie unikālie indeksi tiek saukti par primārajām atslēgām (primary keys) un atsauces uz citu tabulu atslēgām sauc par ārējām atslēgām (foreign keys)  


<!-- ![alt text](https://github.com/adam-p/markdown-here/raw/master/src/common/images/icon48.png "Logo Title Text 1") -->
![alt text](https://techdifferences.com/wp-content/uploads/2017/12/STAR-SCHEMA-EX.jpg "Logo Title Text 1")

Relāciju datubāzes vai arī vispārīgāk, **datubāzes, kuras datus glabā tabulārā formātā atbalsta SQL valodu**
SQL valoda vispārīgas datu manipulācijas ar tabulām vai tabulu šķēlumiem (kas arī ir tabulas).  

Piemērs SQL vaicājumam, kurš parāda visas ienākošās transakcijas 2020. gada janvārī būtu sekojošs:

In [4]:
"""
SELECT 
    Sales.Time_key,
    Sales.Dollars_sold
FROM 
    Sales INNER JOIN TimeDimensionTable on TimeDimensionTable.Time_key = Sales.Time_key
WHERE 
    Month=1 and 
    Year=2020
"""

'\nSELECT \n    Sales.Time_key,\n    Sales.Dollars_sold\nFROM \n    Sales INNER JOIN TimeDimensionTable on TimeDimensionTable.Time_key = Sales.Time_key\nWHERE \n    Month=1 and \n    Year=2020\n'

# Vēl citas problēmas, ko datubāzes risina vai atrisina
* Datubāzēm ir iestrādātas pieejas un atļaujas, tāpat kā Linux operētājsistēmai.  
Ir read/write/alter tiesības, atkarībā no lietotāja. Piem. zvanu centra darbiniekam nav jāredz tehniskās informācijas tabulas, analītiķim nav jāredz visi personiskie dati. Izstrādātājam/testētājam nebūtu jāredz tikai personiskā informācija, bet būtu jāvar izdarīt pilnīgi viss cits.  
* Datubāzes ir iespējams konfigurēt, lai tās datus papildus šifrētu  
* Datubāzēm bieži vien ir iebūvēta 'laika attīšanas' opcija un dublējumi, bet tās ir manuāli jākonfigurē.
* Datubāzes izmanto ACID pieeju attiecībā uz ierakstu atjaunināšanu.  <!-- https://stackoverflow.com/questions/6703897/transaction-handling-how-are-simultaneous-transaction-managed/6704089 -->
* Datubāzes gandrīz vienmēr ierakstus glabā operatīvajā atmiņā, nevis uz diska, kas dod aptuveni 1 - 2 kārtu atšķirību skaitļošanas laika ziņā  
* Datubāzes bieži implementē optimizētus meklēšanas algoritmus (hash table, binary search ir daži piemēri) un optimizētas glabāšanas metodes (datu struktūras tiek glabātas blakus viena otrai atmiņā zināmos attālumos, nevis patvaļīgās vietās)
* Datubāzu datiem viegli var piekļūt izmantojot populārus datubāzu-programmēšanas valodu mijiedarbības standartus (jdbc, odbc), kas atļauj rakstīt ātras aplikācijas un automatizēt lielu skaitu darbību ar datiem


# pandas un MySQL  

Darbu sākam pievēršoties failam MySQL_pandas_API_install.sh
Pēc instrukciju izpildes, ielādējam šos csv ar curl -O komandu terminālī  
https://support.staffbase.com/hc/en-us/article_attachments/360009159392/access-code.csv  
https://support.staffbase.com/hc/en-us/article_attachments/360009159412/access-code-password-recovery-code.csv  
https://support.staffbase.com/hc/en-us/article_attachments/360009197011/username-password-recovery-code.csv  
https://support.staffbase.com/hc/en-us/article_attachments/360009197031/username.csv  
https://support.staffbase.com/hc/en-us/article_attachments/360009197071/email.csv  
https://support.staffbase.com/hc/en-us/article_attachments/360009197051/username-or-email.csv  
https://support.staffbase.com/hc/en-us/article_attachments/360009197091/email-password-recovery-code.csv

Sekojošajam kodam jāpievēršas, kad ir izpildīta Anaconda Python, MySQL, un Python-MySQL API instalācija

In [5]:
class MySQLConnection:
    def __init__(self,user,hostname,db):
        self.user = str(user)
        self.hostname = str(hostname)
        self.db = str(db)
        pword = getpass.getpass("Enter password for user {}".format(user))
        # self.cnx = mysql.connector.connect(host=self.hostname,user=self.user, password=pword,database=self.db)
        self.engine = create_engine("mysql+mysqlconnector://{}:{}@{}/{}".format(user,pword,hostname,db))
        self.cnx = self.engine.connect()
    
    def write_to_db(self, df, table_name):
        try:
            df.to_sql(table_name,con = self.cnx,if_exists='append',index=False)
        except Exception as e:
            print("\n SQL Write error with: ")
            print(df,"\n", e)

In [ ]:
sql = MySQLConnection('ubuntu','13.53.192.184','trenins')

Lasāmviela:  
https://www.bbc.co.uk/bitesize/guides/z8yg87h/revision/4  
https://en.wikipedia.org/wiki/Relational_model  
https://web.stanford.edu/~ouster/cgi-bin/cs142-fall10/lecture.php?topic=sql  
https://www.tutorialspoint.com/CODD-s-Twelve-Rules-of-Relational-Database  
https://en.wikipedia.org/wiki/In-memory_database